In [14]:
import datetime
import logging
import numpy as np
import os
import pandas as pd
import sched
import time
import yfinance as yf

handler_for_errors = logging.FileHandler("errors.log", mode='w')
handler_for_info = logging.FileHandler("logging_de.log", mode='w')
handler_for_errors.setFormatter(logging.Formatter(fmt = "%(asctime)s %(name)s %(levelname)s %(message)s", \
                                                      datefmt = "%Y-%m-%d %H:%M:%S")) 
handler_for_info.setFormatter(logging.Formatter(fmt = "%(asctime)s %(name)s %(levelname)s %(message)s", \
                                                      datefmt = "%Y-%m-%d %H:%M:%S"))
class ErrorFilter(logging.Filter):
    def filter(self, record):
        return record.levelno >= logging.ERROR
handler_for_errors.addFilter(ErrorFilter())

class InfoWarningFilter(logging.Filter):
    def filter(self, record):
        return record.levelno <= logging.WARNING
handler_for_info.addFilter(InfoWarningFilter())

logger = logging.getLogger("logger")
logger.setLevel(logging.DEBUG)
logger.addHandler(handler_for_errors)
logger.addHandler(handler_for_info)

tick_seconds = 2
def current_stock_df_name(stock_name):
    return "current_" + stock_name + ".csv"

def history_stock_df_name(stock_name):
    return "history_" + stock_name + ".csv"

def initialize_file_if_not_exists(filename, file_content):
    if not (os.path.isfile(filename)):
        initialized_file = open(filename, "w")
        initialized_file.write(file_content)
        initialized_file.close()
        
last_request = {}
last_update = {}
    
requests_filename = "current_requests.csv"
stats_filename = "requests_stats.csv"
stats_header = "timestamp,stock,requests\n"
stock_df_header = "Datetime,Open,High,Low,Close,Adj Close,Volume\n"
iteration_id = 0

def update_stocks_collect_stat(scheduler): 
    
    global iteration_id
    global tick_seconds, requests_filename, stats_filename, stats_header, stock_df_header
    global last_request, last_update
    
    import_interval = "5m"
    period_of_import = "60m"
    request_threshold = datetime.timedelta(seconds = 30)
    
    logger.info(f"Iteration {iteration_id} started")
    iteration_timestamp = datetime.datetime.now()
    
    
    logger.debug(f"Checking for new requests")
    if (os.path.isfile(requests_filename)):
        logger.debug(f"New requests detected")
        requests = open(requests_filename, "r")
        requests_heading = requests.readline()
        initialize_file_if_not_exists(stats_filename, stats_header)
        stats = open(stats_filename, "a")
        for raw_row in requests:
            row = raw_row.split(",")
            stock_name = row[0]
            if ((stock_name not in last_request) or \
                (datetime.datetime.now() - last_request[stock_name] > request_threshold)):
                try:
                    logger.debug(f"Start updating {stock_name}")
                    if (os.path.isfile(current_stock_df_name(stock_name))):
                        logger.debug(f"Rewrite {stock_name} current information into history")
                        initialize_file_if_not_exists(history_stock_df_name(stock_name), stock_df_header)
                        history_df = open(history_stock_df_name(stock_name), "a")
                    
                        cur_df = open(current_stock_df_name(stock_name), "r")
                        cur_df.readline()
                        for cur_df_row in cur_df:
                            history_df.write(cur_df_row)
                        cur_df.close()
                        history_df.close()
                        logger.debug(f"{stock_name} current information is saved into history")
                    last_request[stock_name] = datetime.datetime.now()
                    cur_df = yf.download(tickers = stock_name, period = period_of_import, interval = import_interval)
                    cur_df.to_csv(current_stock_df_name(stock_name))
                    logger.debug(f"{stock_name} data updated")
                except Exception as e:
                    logger.error(f"Error {e} occured while getting data")
            stats.write(str(iteration_timestamp) + ",")
            stats.write(raw_row)
        stats.write("\n")
        requests.close()
        stats.close()
        os.remove(requests_filename)
        logger.debug(f"Requests cleared")
    else:
        logger.debug(f"No new requests")
    logger.info(f"Iteration {iteration_id} finished")
    scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (scheduler,))
    iteration_id += 1

my_scheduler = sched.scheduler(time.time, time.sleep)
my_scheduler.enter(tick_seconds, 1, update_stocks_collect_stat, (my_scheduler,))
my_scheduler.run()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


KeyboardInterrupt: 

In [8]:
cur_df = yf.download(tickers = "TSLA", period = "5d", interval = "60m")

[*********************100%%**********************]  1 of 1 completed


In [3]:
import yfinance as yf
stock_data = yf.download(tickers = "AAPL", period ="15m", interval = "1m")

[*********************100%%**********************]  1 of 1 completed


In [10]:
stock_ticker = yf.Ticker("AAPL") 

stock_df = stock_data.history(period = '1d')

if not stock_df.empty: 
    stock_price = stock_df['Close'].iloc[-1] 

print(stock_price)

191.74850463867188
